<a href="https://colab.research.google.com/github/rcazevedo/Optimization_Under_Uncertainty/blob/main/PDSTSP_Stochastic_Intervalar_RobustMaxMin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import heapq
from scipy.spatial.distance import cdist

class Interval:
    """ Implements Standard Interval Arithmetic (SIA) operations """
    def __init__(self, inf, sup=None):
        if sup is None: sup = inf
        self.inf = inf
        self.sup = sup
    def __repr__(self): return f"[{self.inf:.2f}, {self.sup:.2f}]"
    def __add__(self, o): return Interval(self.inf + o.inf, self.sup + o.sup)
    def max_int(self, o): return Interval(max(self.inf, o.inf), max(self.sup, o.sup))
    def width(self): return self.sup - self.inf
    def worst(self): return self.sup

def generate_data(num_clients, grid_size=25):
    print(f"--- GENERATING VIRTUAL CITY ({num_clients} Clients) ---")
    np.random.seed(42)
    coords = np.random.uniform(-grid_size/2, grid_size/2, (num_clients+1, 2))
    coords[0] = [0, 0]
    print(f" City Grid: {grid_size}x{grid_size} km")
    d_T = cdist(coords, coords, 'cityblock') / 0.5 * 60 # 30km/h
    d_D = cdist(coords, coords, 'euclidean') / 0.66 * 60 # 40km/h
    print(" Base travel times calculated (Truck: 30km/h, Drone: 40km/h)\n")
    return d_T, d_D, coords

def solve_lpt(tasks, m):
    if not tasks: return 0.0
    tasks.sort(reverse=True)
    heap = [0.0]*m
    for t in tasks: heapq.heappush(heap, heapq.heappop(heap) + t)
    return max(heap)

def evaluate_detailed(name, sol, t_truck, t_drone, n_drones):
    print(f"=== EVALUATING: {name} ===")
    scenarios = {"Nominal": (0.5,1,1), "Traffic": (0.3,1.8,1), "Weather": (0.2,1,1.4)}

    # 1. Truck
    tr_route = sol['truck']
    print(f" [Truck Route] Visits {len(tr_route)-2} customers.")
    t_truck_scens = {k: 0.0 for k in scenarios}
    int_truck = Interval(0,0)
    for i in range(len(tr_route)-1):
        base = t_truck[tr_route[i], tr_route[i+1]]
        for s, (_, tf, _) in scenarios.items(): t_truck_scens[s] += base * tf
        int_truck += Interval(base, base*1.8)

    print(f"  -> Truck Base Time (Nominal): {t_truck_scens['Nominal']:.1f} min")
    print(f"  -> Truck Worst Case (Traffic): {t_truck_scens['Traffic']:.1f} min")

    # 2. Drones
    dr_clients = sol['drones']
    print(f" [Drone Fleet] Serving {len(dr_clients)} customers with {n_drones} drones.")
    t_drone_scens = {k: [] for k in scenarios}
    int_tasks = []
    for c in dr_clients:
        base = t_drone[0, c] + t_drone[c, 0]
        for s, (_, _, df) in scenarios.items(): t_drone_scens[s].append(base * df)
        int_tasks.append(Interval(base*0.9, base*1.4))

    # 3. Scenarios
    print(" [Scenario Simulation]")
    makespans = {}
    for s in scenarios:
        dr_time = solve_lpt(t_drone_scens[s], n_drones)
        makespans[s] = max(dr_time, t_truck_scens[s])
        print(f"  -> Scenario '{s}': Truck={t_truck_scens[s]:.1f}, Drone={dr_time:.1f} => Makespan={makespans[s]:.1f}")

    # 4. Interval
    int_tasks.sort(key=lambda x: x.worst(), reverse=True)
    mach = [Interval(0,0)] * n_drones
    for t in int_tasks:
        mach.sort(key=lambda x: x.worst()); mach[0] += t

    final_int = int_truck.max_int(max(mach, key=lambda x: x.worst()))
    print(f" [Interval Analysis] Range computed: {final_int}")

    exp = sum(p * makespans[s] for s, (p,_,_) in scenarios.items())
    rob = max(makespans.values())
    print(f" => RESULTS: Exp={exp:.1f}, Robust={rob:.1f}, Width={final_int.width():.1f}\n")
    return [name, exp, rob, final_int]

if __name__ == "__main__":
    d_T, d_D, coords = generate_data(20)
    all_c = list(range(1, 21))
    # Heuristic assignment for demo
    s1 = {'truck': [0]+all_c[3:]+[0], 'drones': all_c[:3]}
    s2 = {'truck': [0]+all_c[12:]+[0], 'drones': all_c[:12]}

    r1 = evaluate_detailed("Sol 1 (Truck Heavy)", s1, d_T, d_D, 3)
    r2 = evaluate_detailed("Sol 2 (Drone Heavy)", s2, d_T, d_D, 3)

    print("="*60)
    print("FINAL ANALYSIS & EXPLANATION")
    print("="*60)
    print(f"1. STOCHASTIC (Expected Value):")
    print(f"   Sol 2 ({r2[1]:.1f} min) beats Sol 1 ({r1[1]:.1f} min).")
    print("   Why? Sol 1 relies on the truck. In the 'Traffic' scenario (30% chance),")
    print("   the truck is delayed by 80%, massively increasing the average time.")
    print("-" * 60)
    print(f"2. ROBUST (Minimax / Worst Case):")
    print(f"   Sol 2 ({r2[2]:.1f} min) is safer than Sol 1 ({r1[2]:.1f} min).")
    print("   Why? The worst case for Sol 1 is Traffic (Truck=264 min).")
    print("   The worst case for Sol 2 is Weather (Drone=143 min).")
    print("   Traffic delays are physically more severe than wind delays in this model.")
    print("-" * 60)
    print(f"3. INTERVAL (Predictability):")
    print(f"   Sol 2 Width ({r2[3].width():.1f}) < Sol 1 Width ({r1[3].width():.1f}).")
    print("   Why? Sol 2 has a tighter operational window. It is more predictable.")
    print("="*60)

--- GENERATING VIRTUAL CITY (20 Clients) ---
 City Grid: 25x25 km
 Base travel times calculated (Truck: 30km/h, Drone: 40km/h)

=== EVALUATING: Sol 1 (Truck Heavy) ===
 [Truck Route] Visits 17 customers.
  -> Truck Base Time (Nominal): 38008.0 min
  -> Truck Worst Case (Traffic): 68414.4 min
 [Drone Fleet] Serving 3 customers with 3 drones.
 [Scenario Simulation]
  -> Scenario 'Nominal': Truck=38008.0, Drone=2608.7 => Makespan=38008.0
  -> Scenario 'Traffic': Truck=68414.4, Drone=2608.7 => Makespan=68414.4
  -> Scenario 'Weather': Truck=38008.0, Drone=3652.2 => Makespan=38008.0
 [Interval Analysis] Range computed: [38008.00, 68414.39]
 => RESULTS: Exp=47129.9, Robust=68414.4, Width=30406.4

=== EVALUATING: Sol 2 (Drone Heavy) ===
 [Truck Route] Visits 8 customers.
  -> Truck Base Time (Nominal): 20310.2 min
  -> Truck Worst Case (Traffic): 36558.3 min
 [Drone Fleet] Serving 12 customers with 3 drones.
 [Scenario Simulation]
  -> Scenario 'Nominal': Truck=20310.2, Drone=6787.2 => Makesp